# Parameter definition
Choose all the settings for the database creation.

In [1]:
db_name= 'test_video_dataset' #'colab_main_dataset'
database_names=['enfsi2015'] # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
detector_names=['mtcnn_serfiq'] # 'dlib', 'mtcnn'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = [] # 'confusion_score',
save_in_drive = False # To work in google colab

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [2]:
from sql_face.sqldb import SQLDataBase

In [3]:
database = SQLDataBase(db_name=db_name)

Creating Db file at /home/andrea/Desktop/sql_database/test_video_dataset.db


NameError: name 'Base' is not defined

In [ ]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/andrea/Desktop/image_datasets'}